In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/cs688project2/data/"
dev_seen_data= pd.read_json(path+'dev_seen.jsonl',lines=True)
dev_unseen_data= pd.read_json(path+'dev_unseen.jsonl',lines=True)
test_seen_data=pd.read_json(path+'test_seen.jsonl',lines=True)
test_unseen_data=pd.read_json(path+'test_unseen.jsonl',lines=True)
train_data=pd.read_json(path+'train.jsonl',lines=True)

In [ ]:
data = pd.concat([train_data,test_seen_data,test_unseen_data,dev_seen_data,dev_unseen_data])
data1 = data.drop_duplicates(subset=['img'])

In [ ]:
!pip install google-cloud-vision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 9.3 MB/s eta 0:00:00


In [ ]:
from google.cloud import vision

In [ ]:
#set environment variable to pick up google credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/drive/MyDrive/myproject-svyerra-40d2858a7beb.json"

In [ ]:
def detect_web(path):
    """Detects web annotations given an image."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.web_detection(image=image)
    annotations = response.web_detection

    output = []
    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            output.append(label.label)

    if annotations.web_entities:
        for entity in annotations.web_entities[:4]:
            output.append(entity.description)

    return output

In [ ]:
path = '/content/drive/MyDrive/cs688project2/data/'
output_entities = {}
count=0
for each in data1['img']:
  if each not in done:
    img_path = path + each
    output_entities[each] = detect_web(img_path)
    print(count)
    count+=1
    print(output_entities[each])

In [ ]:
data1['entities'] = np.nan

In [ ]:
df22 = pd.DataFrame.from_dict(output_entities, orient="index")
df22.reset_index(inplace=True)


In [ ]:
final_df = data1.merge(df22, left_on='img', right_on='index')
final_df.rename(columns={0: "0", 1: "1",2: "2",3: "3",4: "4",}, inplace=True)
final_df.drop('index', inplace=True, axis=1)
final_df['combined_text'] = final_df['text']+' ' +final_df['0'] + ' ' + final_df['1'] + ' ' + final_df['2'] + ' ' + final_df['3'] + ' ' + final_df['4']
final_df.to_csv('/content/drive/MyDrive/cs688project2/data/final_cleaned.csv')
